In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from config import db_password
from sqlalchemy import create_engine
import psycopg2
import pandas.io.sql as sqlio
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\f

In [2]:
#create database connection variable 
conn = psycopg2.connect(user="postgres", password=db_password, host="localhost", database="final_project")

In [3]:
#execute query and save it to a variable
query="select * from chess_data"
chess_df = sqlio.read_sql_query(query,conn)

In [4]:
chess_df.drop(['id'],axis=1, inplace=True)

In [5]:
# Split moves column into moves df
moves_df = chess_df['moves'].str.split(' ', n=10, expand=True)

# Drop column 10 and rename columns
moves_df=moves_df.drop(10,axis=1)
moves_df.columns= ["Wm1","Bm1","Wm2","Bm2","Wm3","Bm3","Wm4","Bm4","Wm5","Bm5"]

moves_df["outcome"] = chess_df["winner"]

# drop na
moves_df = moves_df.dropna()

In [6]:
# Changing moves to numbers
for col in moves_df.columns:
    print(col,end=' ')
    
    # Get list of unique values
    values = list(set(moves_df[col].values))
    
    # Create numerical dictionary
    values_with_indexes = {}
    for i, v in enumerate(values):
        values_with_indexes[v] = i
    
    # Replace column
    moves_df.replace({col: values_with_indexes},inplace=True)

moves_df.head()

Wm1 Bm1 Wm2 Bm2 Wm3 Bm3 Wm4 Bm4 Wm5 Bm5 outcome 

,Wm1,Bm1,Wm2,Bm2,Wm3,Bm3,Wm4,Bm4,Wm5,Bm5,outcome
0,4,3,45,21,270,323,460,241,313,610,1
1,4,2,94,46,191,124,185,315,362,153,3
2,19,6,70,135,180,196,19,56,13,550,1
3,4,3,29,108,142,297,419,68,18,214,1
4,19,6,29,135,156,116,228,149,560,416,1


In [7]:
# Export csv containing sample data to be imported into ML
# moves_df.to_csv("ML_sample_data.csv")

In [8]:
# Split our preprocessed data into our features and target arrays
y = moves_df["outcome"].values
X = moves_df.drop("outcome",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1)

C:\Users\mulle\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

## First model - one feature layer (sigmoid)

In [11]:
# Create the keras sequential model
number_input_features = len(X_train_scaled[0])
nn_model = tf.keras.models.Sequential()

In [12]:
# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim = number_input_features))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [14]:
# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 11        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
754754/754754 [==============================] - 9s 12us/sample - loss: -20.3681 - acc: 0.4970
Epoch 2/5
754754/754754 [==============================] - 10s 13us/sample - loss: -61.5234 - acc: 0.4972
Epoch 3/5
754754/754754 [==============================] - 9s 12us/sample - loss: -102.6135 - acc: 0.4972
Epoch 4/5
754754/754754 [==============================] - 9s 11us/sample - loss: -143.7388 - acc: 0.4972
Epoch 5/5
754754/754754 [==============================] - 9s 11us/sample - loss: -184.7978 - acc: 0.4972


In [17]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251585/251585 - 1s - loss: -2.0471e+02 - acc: 0.4992
Loss: -204.71498644637919, Accuracy: 0.49915534257888794


In [18]:
# Train the model with more epochs
fit_model_b = nn_model.fit(X_train_scaled, y_train, epochs=15)

Epoch 1/15
754754/754754 [==============================] - 9s 12us/sample - loss: -225.8812 - acc: 0.4972
Epoch 2/15
754754/754754 [==============================] - 9s 11us/sample - loss: -266.9584 - acc: 0.4972
Epoch 3/15
754754/754754 [==============================] - 9s 11us/sample - loss: -308.0471 - acc: 0.4972
Epoch 4/15
754754/754754 [==============================] - 9s 12us/sample - loss: -349.1428 - acc: 0.4972
Epoch 5/15
754754/754754 [==============================] - 9s 12us/sample - loss: -390.2609 - acc: 0.4972
Epoch 6/15
754754/754754 [==============================] - 9s 12us/sample - loss: -431.3306 - acc: 0.4972
Epoch 7/15
754754/754754 [==============================] - 9s 12us/sample - loss: -472.4438 - acc: 0.4972
Epoch 8/15
754754/754754 [==============================] - 9s 12us/sample - loss: -513.5204 - acc: 0.4972
Epoch 9/15
754754/754754 [==============================] - 9s 12us/sample - loss: -554.6035 - acc: 0.4972
Epoch 10/15
754754/754754 [==========

In [19]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251585/251585 - 1s - loss: -8.1917e+02 - acc: 0.4992
Loss: -819.1716913964683, Accuracy: 0.49915534257888794


## Second model - one feature layer (relu)

In [20]:
# Create the keras sequential model
nn_model_2 = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim = number_input_features))

# Add the ouput layer that uses a probability activation function
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Create a summary to check the structure of the sequential model
nn_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 11        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model and customize metrics
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model_2 = nn_model_2.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
754754/754754 [==============================] - 10s 13us/sample - loss: -279.7402 - acc: 0.4972
Epoch 2/5
754754/754754 [==============================] - 10s 13us/sample - loss: -1698.0329 - acc: 0.4972
Epoch 3/5
754754/754754 [==============================] - 10s 13us/sample - loss: -4366.9813 - acc: 0.4972
Epoch 4/5
754754/754754 [==============================] - 9s 13us/sample - loss: -8283.6514 - acc: 0.4972
Epoch 5/5
754754/754754 [==============================] - 9s 12us/sample - loss: -13441.4359 - acc: 0.4972


In [24]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
#predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_model_2.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

251585/251585 - 1s - loss: -3.8328e+04 - acc: 0.4992
Loss: -38328.0528272282, Accuracy: 0.49915534257888794


## Third model - two feature layers (relu + relu) and more nodes

In [28]:
# Create the keras sequential model
nn_model_3 = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model_3.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim = number_input_features))

# Add the second input layer
nn_model_3.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Add the ouput layer that uses a probability activation function
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Create a summary to check the structure of the sequential model
nn_model_3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 6         
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model and customize metrics
nn_model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model_3 = nn_model_3.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
754754/754754 [==============================] - 10s 14us/sample - loss: -73604.9132 - acc: 0.4969
Epoch 2/5
754754/754754 [==============================] - 10s 14us/sample - loss: -832656.0971 - acc: 0.4972
Epoch 3/5
754754/754754 [==============================] - 10s 14us/sample - loss: -3175532.1558 - acc: 0.4972
Epoch 4/5
754754/754754 [==============================] - 11s 14us/sample - loss: -7983189.5394 - acc: 0.4972
Epoch 5/5
754754/754754 [==============================] - 11s 14us/sample - loss: -16118421.7438 - acc: 0.4972


In [31]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
#predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn_model_3.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

251585/251585 - 2s - loss: -4.4866e+09 - acc: 0.4992
Loss: -4486617857.9425, Accuracy: 0.49915534257888794


## fourth model - same as above with new training/testing data

In [32]:
# Split the preprocessed data into a training and testing dataset
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X,y,random_state = 1)

# Create a StandardScaler instances
scaler_2 = StandardScaler()

# Fit the StandardScaler
X_scaler_2 = scaler_2.fit(X_train_2)

# Scale the data
X_train_scaled_2 = X_scaler_2.transform(X_train_2)
X_test_scaled_2 = X_scaler_2.transform(X_test_2)

In [33]:
# Create the keras sequential model
nn_model_4 = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model_4.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim = number_input_features))

# Add the second input layer
nn_model_4.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Add the ouput layer that uses a probability activation function
nn_model_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Create a summary to check the structure of the sequential model
nn_model_4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6         
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model and customize metrics
nn_model_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_model_4 = nn_model_4.fit(X_train_scaled_2, y_train_2, epochs=5)

Epoch 1/5
754754/754754 [==============================] - 10s 14us/sample - loss: -142808.9896 - acc: 0.4970
Epoch 2/5
754754/754754 [==============================] - 10s 14us/sample - loss: -1622216.7928 - acc: 0.4972
Epoch 3/5
754754/754754 [==============================] - 11s 14us/sample - loss: -6220159.7885 - acc: 0.4972
Epoch 4/5
754754/754754 [==============================] - 11s 14us/sample - loss: -15660132.9440 - acc: 0.4972
Epoch 5/5
754754/754754 [==============================] - 11s 14us/sample - loss: -31656638.7923 - acc: 0.4972


In [36]:
4# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
#predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss_4, model_accuracy_4 = nn_model_3.evaluate(X_test_2,y_test_2,verbose=2)
print(f"Loss: {model_loss_4}, Accuracy: {model_accuracy_4}")

251585/251585 - 2s - loss: -4.4866e+09 - acc: 0.4992
Loss: -4486617857.9425, Accuracy: 0.49915534257888794


**note to talk to group about** - accuracy score is constant across all models, loss keeps growing. is something being initalized improperly?

tried using a different set of testing/training data but accuracy remained the same